# Deep Learning Project

Developed by: Mandy Sack
August 2019


In [ ]:
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import sklearn
import sys
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [ ]:
# import the data
#first, verify you have the files in the correct location
print(os.path.isdir("social_honeypot_icwsm_2011"))
print(os.path.isfile("social_honeypot_icwsm_2011\content_polluters_classified.csv"))
print(os.path.isfile("social_honeypot_icwsm_2011\legitimate_users_classified.csv"))

bDcolnames=['UserID','CreatedAt','CollectedAt', 'NumberOfFollowings', 'NumberOfFollowers', 'NumberOfTweets','LengthOfScreenName','LengthOfDescriptionInUserProfile','BadUser'] 
badData = pd.read_csv("social_honeypot_icwsm_2011\content_polluters_classified.csv", names=bDcolnames)

nDcolnames=['UserID','CreatedAt','CollectedAt', 'NumberOfFollowings', 'NumberOfFollowers', 'NumberOfTweets','LengthOfScreenName','LengthOfDescriptionInUserProfile','BadUser']
normalData = pd.read_csv("social_honeypot_icwsm_2011\legitimate_users_classified.csv", names=nDcolnames)

#TODO: make this configurable to enable your own data to be passed in 

In [ ]:
print(badData.head(3))
print(normalData.head(3))

In [ ]:
#insert classification of 0 or 1 based on if good or bad user
#TODO- this does not change accurately once the two datasets are merged
#badData.insert(8, "Bad User", 1, True) 
#print(badData.head(3))

#normalData.insert(8,"Bad User",0, True)
#print(normalData.head(3))

In [ ]:
#set the Collected column to a datetime object 
#badData.CollectedAt = pd.to_datetime(badData.CollectedAt)
#normalData.CollectedAt = pd.to_datetime(normalData.CollectedAt)

In [ ]:
#merge the data together for 1 dataset based on CollectedAt time & save to file
#mergedData = pd.merge_ordered(badData, normalData, fill_method='ffill', on='CollectedAt')
#mergedData = pd.merge_ordered(badData, normalData, fill_method='ffill', left_on='CollectedAt', right_on='CollectedAt')
mergedData = pd.concat([badData,normalData])
mergedData.sort_values(by=['CollectedAt'], inplace=True)
mergedData.to_csv("social_honeypot_icwsm_2011\mergedData_classified.csv")
mergedData

In [ ]:
#Explore badData dataset seperately
print("badData shape")
print(format(badData.shape))
print("\n")
print("badData NumberOfFollowings describe")
print(badData['NumberOfFollowings'].describe())
print("\n")
print("badData NumberOfFollowers describe")
print(badData['NumberOfFollowers'].describe())
print("\n")
print("badData NumberOfTweets describe")
print(badData['NumberOfTweets'].describe())
print("\n")
print("badData LengthOfScreenName describe")
print(badData['LengthOfScreenName'].describe())
print("\n")
print("badData LengthOfDescriptionInUserProfile describe")
print(badData['LengthOfDescriptionInUserProfile'].describe())

In [ ]:
#Explore normalData dataset seperately
print("normalData shape")
print(format(normalData.shape))
print("\n")
print("normalDataData NumberOfFollowings describe")
print(normalData['NumberOfFollowings'].describe())
print("\n")
print("normalDataData NumberOfFollowers describe")
print(normalData['NumberOfFollowers'].describe())
print("\n")
print("normalDataData NumberOfTweets describe")
print(normalData['NumberOfTweets'].describe())
print("\n")
print("normalDataData LengthOfScreenName describe")
print(normalData['LengthOfScreenName'].describe())
print("\n")
print("normalDataData LengthOfDescriptionInUserProfile describe")
print(normalData['LengthOfDescriptionInUserProfile'].describe())

In [ ]:
#Explore mergedData dataset seperately
print("mergedData shape")
print(format(mergedData.shape))
print("\n")
print("mergedData NumberOfFollowings describe")
print(mergedData['NumberOfFollowings'].describe())
print("\n")
print("mergedData NumberOfFollowers describe")
print(mergedData['NumberOfFollowers'].describe())
print("\n")
print("mergedData NumberOfTweets describe")
print(mergedData['NumberOfTweets'].describe())
print("\n")
print("mergedData LengthOfScreenName describe")
print(mergedData['LengthOfScreenName'].describe())
print("\n")
print("mergedData LengthOfDescriptionInUserProfile describe")
print(mergedData['LengthOfDescriptionInUserProfile'].describe())

In [ ]:
#TODO Merge together the columns of features for plotting
#Let's plot features to determine if anything stands out

In [ ]:
#Remove nonnumeric columns
mergedData.drop("CreatedAt", axis=1, inplace=True)
mergedData.drop("CollectedAt", axis=1, inplace=True)
mergedData.drop("UserID", axis=1, inplace=True)
mergedData.head(3)

In [ ]:
#normalize features
#mergedDataNorm = mergedData.apply(lambda x: (x-x.min()) / (x.max() - x.min()))
#mergedDataNorm

In [ ]:
#create feaure columns from our numerical data
#numOfFlg = tf.feature_column.numeric_column("NumberOfFollowings")
numOfFlr = tf.feature_column.numeric_column("NumberOfFollowers")
numOfTw = tf.feature_column.numeric_column("NumberOfTweets")
lenScrNm = tf.feature_column.numeric_column("LengthOfScreenName")
lenDsPro = tf.feature_column.numeric_column("LengthOfDescriptionInUserProfile")

numOfFlr = tf.feature_column.numeric_column('NumberOfFollowings',
                    normalizer_fn=lambda x: x - mergedData.describe().mean())
feat_columns=[numOfFlr,numOfTw,lenScrNm,lenDsPro,numOfFlr]

In [ ]:
#estimator = tf.estimator.LinearClassifer(feature_columns=[numOfFlr,numOfTw,lenScrNm,lenDsPro,numOfFlr])

In [ ]:
#estimator.train(input_fn=mergedData,steps=2000)

In [ ]:
bdNFlr_mean= badData.mean()
print(bdNFlr_mean)

In [ ]:
column_names = list(mergedData.columns)

feature_names = column_names[:-1]
label_name = column_names[-1]

print("Features: {}".format(feature_names))
print("Label: {}".format(label_name))

In [ ]:
class_names = ['bot', 'not bot']

In [ ]:
#can try this again with version 2.25 pandas
#mergedData.to_csv("social_honeypot_icwsm_2011\trainingData_classified.csv")

In [ ]:
#batch_size = 32
#train_dataset_fp = tf.keras.utils.get_file(fname="social_honeypot_icwsm_2011\mergedData_classified.csv")
#train_dataset = tf.contrib.data.make_csv_dataset(
#    mergedData,
#    batch_size,
#    column_names=column_names,
#    label_name=label_name,
#    num_epochs=1)

In [ ]:
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
train, test = train_test_split(mergedData, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')
print(dict(mergedData), 'dict(mergedData)')
#print(list(feature_batch.keys()), 'feature_batch.keys()')

In [ ]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('BadUser')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [ ]:
batch_size = 32 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size)
print(train_ds, "train_ds")
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
print(val_ds, "val_ds")
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)
print(test_ds, "test_ds")

In [ ]:
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of NumberOfFollowings:', feature_batch['NumberOfFollowings'])
  print('A batch of Bad Users:', label_batch )

In [ ]:
# A utility method to create a feature column
# and to transform a batch of data
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())

In [ ]:
# We will use this batch to demonstrate several types of feature columns
example_batch = next(iter(train_ds))[0]

In [ ]:
feature_columns.append(feature_column.numeric_column(mergedData.columns))

In [ ]:
numOfFors = feature_column.numeric_column("NumberOfFollowers")
demo(numOfFors)

In [ ]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [ ]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [ ]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'],
              run_eagerly=True)

model.fit(train_ds,
          validation_data=val_ds,
          epochs=5)

In [ ]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)